<a href="https://colab.research.google.com/github/DJongstra/Information_Retrieval_Assignment_3/blob/main/IR_PlagiarismDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all needed libraries


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

Drive mount

In [ ]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

Read the data of the small news article set

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IR-Assignment-3/data/news_articles_small.csv', index_col=0)
print(df.head())

All the articles in the small article dataset will be processed to a list of the terms in the articles. The words are lowercased and duplicates are removed by using a set.

In [ ]:
articleList = []

for index, row in df.iterrows():
    temp = (row['article'].lower().split())
    temp = set(temp)
    articleList.append(temp)
    
print(articleList[0])

Calculate the jaccard index between each two documents in the data set by dividing the length of the intersection with the length of the union of the two sets. Save the values to a list to use later.

In [ ]:
jaccardVals = []

for doc1idx in range(len(articleList)):
  doc1 = articleList[doc1idx]
  doc2idx = doc1idx + 1
  while doc2idx < len(articleList):
    doc2 = articleList[doc2idx]
    jaccard = len(doc1.intersection(doc2)) / len(doc1.union(doc2))
    jaccardVals.append(jaccard)
    doc2idx += 1

Plot the amount of values per bin, using a total of 50 bins.


In [ ]:
jaccardVals = np.array(jaccardVals)
sns.histplot(jaccardVals, bins=50)


The previous graph showed a peak in a small range of the possible similarities. To see the distribution in other ranges, we leave the peak values out.

From this it is clear that there are also values in the higher ranges, however there are not a lot.

In [ ]:
sns.histplot(jaccardVals[jaccardVals>0.2], bins=40)

# 2. Preprocessing of data, shingling, and minhashing to generate a signature matrix using news articles small.csv dataset.

import libraries

In [ ]:
!pip install mmh3
!pip install snapy
import string, re
from snapy import MinHash, LSH

get content

In [ ]:
articleList = []

for index, row in df.iterrows():
  #News_ID = int(row['News_ID']) # id
  article = row['article'] # lower case
  #article = article.lower() # lower case
  #article = article.replace("n't", " not").replace("'ve", " have").replace("'s","") # rewrite contractions
  #article = re.sub(" [^ ]*&amp[^ ]*","", article) # remove random "&amp"'s in text
  #article = article.translate(str.maketrans('', '', string.digits)) # remove numbers?
  #article = re.sub(" +"," ", article) # remove double spaces
  #article = article.translate(str.maketrans('', '', string.punctuation)) # remove ALL punctuation
  articleList.append(article)

print(articleList[0])

In [ ]:
# Create MinHash object.
minhash = MinHash(articleList, n_gram=4, permutations=50, hash_bits=64, seed=3)

In [ ]:
# Create LSH model.
lsh = LSH(minhash, range(len(articleList)), no_of_bands=50)

In [ ]:
results = lsh.edge_list(min_jaccard=0.8, jaccard_weighted=True)

In [ ]:
#for label,near_duplicates in lsh.adjacency_list(min_jaccard=0.8).items():
#  if near_duplicates:
 #   print("News_id", label, "has near duplicates", near_duplicates)

print("DOC1", "DOC2", "JACCARD")
for doc1_id ,doc2_id, jaccardVal in results:
  print(doc1_id ,"",doc2_id, "", jaccardVal)